In [ ]:
%reload_ext autoreload
%autoreload 2
import matplotlib
# matplotlib.use('TkAgg')
# %matplotlib
%config Completer.use_jedi = False

In [ ]:
from causal_narrative.fmriutil import *
from causal_narrative.datautil import *

## Cross Layer Causal

In [ ]:
insert_layer=3
check_layer=8
# model_type="opt-125m"
# model_type="opt-350m"
model_type="gpt2"

fu = FmriUtil()
ndp = NarrativeDataPreprocess()
resdict_l = []
for ii_trial in tqdm_notebook(range(10)):
    print("Trial number:",ii_trial)
    resdict={}
    for task in ndp.tasks:
        gptcodeall, gptcodeall_noise, noise_paral = fu.opt_encode_causalstudy(task=task, insert_layer=insert_layer, model_type=model_type)
        resdict[task]=gptcodeall[check_layer,:,:], gptcodeall_noise[check_layer,:,:], noise_paral
    resdict_l.append(resdict)

In [ ]:
ndp = NarrativeDataPreprocess()
data_home = "../dataset/narrative_encode"
opt_model = "opt-125m_no_case"
# opt_model = "opt-350m"
# opt_model = "gpt2"
window = 128
layer=insert_layer
featurel = []
for task in ndp.tasks:
    mat = load_data(os.path.join(data_home, "%s/opt_no_case_alignmat.pickle" % task))
    feature_all = load_data(
        os.path.join(data_home, "%s/learning_dynamics/opt_encode_%s_w%s.data" % (task, opt_model, window)))
#     mat = load_data(os.path.join(data_home, "%s/opt_alignmat.pickle" % task))
#     mat = load_data(os.path.join(data_home, "%s/gpt2_alignmat.pickle" % task))
#     feature_all = load_data(
#         os.path.join(data_home, "%s/opt_encode_%s_w%s.data" % (task, opt_model, window)))
    feature = feature_all[layer, :, :].type(torch.FloatTensor).numpy()
    featurel.append(feature)
feature_all3 = np.concatenate(featurel, axis=0)

In [ ]:
feature_pca3, pM3, V3 = pca_proj(feature_all3.T, 20)
feature_pca3 = feature_pca3.T

In [ ]:
layer=check_layer
featurel = []
for task in ndp.tasks:
    mat = load_data(os.path.join(data_home, "%s/opt_no_case_alignmat.pickle" % task))
    feature_all = load_data(
        os.path.join(data_home, "%s/learning_dynamics/opt_encode_%s_w%s.data" % (task, opt_model, window)))
#     mat = load_data(os.path.join(data_home, "%s/opt_alignmat.pickle" % task))
#     mat = load_data(os.path.join(data_home, "%s/gpt2_alignmat.pickle" % task))
#     feature_all = load_data(
#         os.path.join(data_home, "%s/opt_encode_%s_w%s.data" % (task, opt_model, window)))
    feature = feature_all[layer, :, :].type(torch.FloatTensor).numpy()
    featurel.append(feature)
feature_all8 = np.concatenate(featurel, axis=0)

In [ ]:
feature_pca8, pM8, V8 = pca_proj(feature_all8.T, 20)
feature_pca8 = feature_pca8.T

In [ ]:
gptcodeall_l = []
gptcodeall_noise_l = []
noise_all_l = []
for resdict in resdict_l:
    for key in resdict.keys():
        gptcodeall, gptcodeall_noise, noise_paral = resdict[key]
        noise_all = []
        for para in noise_paral:
            noise_all.append(para['add_noise'].numpy())
        noise_all=np.array(noise_all)
        gptcodeall_l.append(gptcodeall.numpy())
        gptcodeall_noise_l.append(gptcodeall_noise.numpy())
        noise_all_l.append(noise_all)
gptcodeall_l = np.concatenate(gptcodeall_l,axis=0)
gptcodeall_noise_l = np.concatenate(gptcodeall_noise_l,axis=0)
noise_all_l = np.concatenate(noise_all_l,axis=0)

In [ ]:
gptcodeall_pca = gptcodeall_l.dot(pM8.T)
gptcodeall_noise_pca = gptcodeall_noise_l.dot(pM8.T)
diff_pca = gptcodeall_noise_pca-gptcodeall_pca

noise_pca = noise_all_l.dot(pM3.T)

In [ ]:
_, t_re = fit_linear(diff_pca[:,:], noise_pca[:,:])
transformation_matrix, bias_vector = t_re

In [ ]:
# Visualize the matrix
# plt.imshow(transformation_matrix.T, cmap='seismic')
trasn_mat = transformation_matrix.T
plt.imshow(np.abs(trasn_mat), cmap='OrRd', vmin=0.0, vmax=0.3)
plt.xlabel("Layer 9 indexes",size=12)
plt.ylabel("Layer 4 indexes",size=12)
plt.colorbar()
plt.show()

In [ ]:
thred = np.median(np.abs(trasn_mat))
thred

In [ ]:
cause_mat = np.zeros((20,20))
for ii in range(20):
    for jj in range(20):
        if np.abs(trasn_mat)[ii,jj]>thred:
            cause_mat[ii,jj]=1

In [ ]:
# Visualize the matrix
plt.imshow(cause_mat, cmap='seismic', vmin=-1.5, vmax=1.5)
plt.colorbar()
plt.show()

In [ ]:
sinkdeg = np.sum(cause_mat, axis=0)
sinkdeg

In [ ]:
sorted_indices = np.argsort(sinkdeg)
sorted_indices

In [ ]:
bottom_index = [0,  2,  1,  6,  7,  3, 17, 10,  4,  8]
top_index = [15, 12, 19,  5, 14, 18, 16, 9, 11, 13]

In [ ]:
layer=check_layer

for task in ndp.tasks:
#     mat = load_data(os.path.join(data_home, "%s/opt_no_case_alignmat.pickle" % task))
#     feature_all = load_data(
#         os.path.join(data_home, "%s/learning_dynamics/opt_encode_%s_w%s.data" % (task, opt_model, window)))
#     mat = load_data(os.path.join(data_home, "%s/opt_alignmat.pickle" % task))
    mat = load_data(os.path.join(data_home, "%s/gpt2_alignmat.pickle" % task))
    feature_all = load_data(
        os.path.join(data_home, "%s/opt_encode_%s_w%s.data" % (task, opt_model, window)))
    feature= feature_all[layer, :, :].type(torch.FloatTensor).numpy()
    
    feature_pca = feature.dot(pM8.T)
    feature_pick = feature_pca[:,top_index]
    
    feature_alg = mat.T.dot(feature_pick)

    zero_mask = (feature_alg != 0)[:, 0]
    zero_feature = ((feature_alg == 0)[:, 0]).astype(int).reshape(-1, 1)
    zero_feature = scipy.stats.zscore(zero_feature)
    
    feature_alg[zero_mask, :] = scipy.stats.zscore(feature_alg[zero_mask, :])
    feature_alg_zero = np.concatenate([zero_feature, feature_alg], axis=-1)
    feature_alg_zero = scipy.stats.zscore(feature_alg_zero)  # mat_pca_zscoresep # not big deal
    save_data((feature_alg_zero, zero_mask), 
              os.path.join(data_home, "%s/multilayer_4c9_gpt2_top/opt_encode_%s_l%s.data" % (task, opt_model, layer)))